In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Summarization with Generative Models on Vertex AI

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_summarization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_summarization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/language/examples/prompt-design/text_summarization.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview
Text summarization produces a concise and fluent summary of a longer text document. There are two main text summarization types: extractive and abstractive. Extractive summarization involves selecting critical sentences from the original text and combining them to form a summary. Abstractive summarization involves generating new sentences representing the original text's main points. In this notebook, you go through a few examples of how large language models can help with generating summaries based on text.

Learn more about text summarization in the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/text/summarization-prompts).

### Objective

In this tutorial, you will learn how to use generative models to summarize information from text by working through the following examples:
- Transcript summarization
- Summarizing text into bullet points
- Dialogue summarization with to-dos
- Hashtag tokenization
- Title & heading generation

You also learn how to evaluate model-generated summaries by comparing to human-created summaries using ROUGE as an evaluation framework.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Generative AI Studio

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Getting Started

### Install Vertex AI SDK

In [ ]:
!pip install google-cloud-aiplatform google-cloud-translate --upgrade --user

**Colab only:** Uncomment the following cell to restart the kernel or use the button to restart the kernel. For Vertex AI Workbench you can restart the terminal using the button on top. 

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Authenticating your notebook environment
* If you are using **Colab** to run this notebook, uncomment the cell below and continue.
* If you are using **Vertex AI Workbench**, check out the setup instructions [here](#TODO).
* If you are using **local Jupyter**, check out the setup instructions [here](#TODO).

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

Let's start by importing the libraries that we will need for this tutorial

**Colab only:** Uncomment the following cell to initialize the Vertex AI SDK. For Vertex AI Workbench, you don't need to run this.  

In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
from vertexai.preview.language_models import TextGenerationModel

### Import models

Here we load the pre-trained text generation model called `text-bison@001`.

In [ ]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

### Create the translation wrapper function

In [ ]:
from google.cloud import translate

project_id = !gcloud config list project
project_id = project_id[1].split('=')[1].strip()
parent = f'projects/' + project_id


def traduza(texto, idioma_destino):
    client = translate.TranslationServiceClient()

    response = client.translate_text(
        parent=parent,
        contents=[texto],
        target_language_code=idioma_destino,
        mime_type="text/plain"
    )

    return response.translations[0].translated_text

## Text Summarization

### Transcript summarization

In this first example, you summarize a piece of text on quantum computing.

In [ ]:
prompt = traduza("""
Forneça um resumo muito curto, não mais do que três frases, para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

Resumo:

""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "pt")
)

Instead of a summary, we can ask for a TL;DR ("too long; didn't read"). You can compare the differences between the outputs generated.

In [ ]:
prompt = traduza("""
Forneça um TL;DR para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

TL;DR:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=1024, top_k=40, top_p=0.8).text, "pt")
)

### Summarize text into bullet points
In the following example, you use same text on quantum computing, but ask the model to summarize it in bullet-point form. Feel free to change the prompt.

In [ ]:
prompt = traduza("""
Forneça um resumo muito curto em quatro bullets para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

Bullets:

""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=1, top_p=0.8).text, "pt")
)

###  Dialogue summarization with to-dos
Dialogue summarization involves condensing a conversation into a shorter format so that you don't need to read the whole discussion but can leverage a summary. In this example, you ask the model to summarize an example conversation between an online retail customer and a support agent, and include to-dos at the end.

In [ ]:
prompt = traduza("""
Gere um resumo da conversa a seguir e, no final, resuma as tarefas para o agente de suporte:

Cliente: Olá, sou José e recebi o item errado.

Agente de suporte: Olá, José. Como você gostaria de ver isso resolvido?

Cliente: Eu quero devolver o item e obter um reembolso, por favor.

Agente de Suporte: Claro. Posso processar o reembolso para você agora. Posso ter o número do seu pedido, por favor?

Cliente: É [NÚMERO DO PEDIDO].

Agente de suporte: Obrigado. Processei o reembolso e você receberá seu dinheiro de volta em 14 dias.

Cliente: Muito obrigado.

Agente de suporte: De nada, José. Tenha um bom dia!

Resumo:
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.2, max_output_tokens=256, top_k=40, top_p=0.8).text, "pt")
)

###  Hashtag tokenization
Hashtag tokenization is the process of taking a piece of text and getting the hashtag "tokens" out of it. You can use this, for example, if you want to generate hashtags for your social media campaigns. In this example, you take [this tweet from Google Cloud](https://twitter.com/googlecloud/status/1649127992348606469) and generate some hashtags you can use.

In [ ]:
prompt = traduza("""
Tokenize as hashtags deste tweet:

Google Cloud
@googlecloud
Como os dados podem ajudar nosso planeta em mudança? 🌎

Em homenagem ao #EarthDay neste fim de semana, estamos orgulhosos de compartilhar como estamos fazendo parceria com
@ClimateEngine para aproveitar o poder dos dados geoespaciais e direcionar para um futuro mais sustentável.

Confira como → https://goo.gle/3mOUfts
""", "en")

print(
    generation_model.predict(prompt, temperature=0.8, max_output_tokens=1024, top_k=40, top_p=0.8).text
)

### Title & heading generation
Below, you ask the model to generate five options for possible title/heading combos for a given piece of text.

In [ ]:
prompt = traduza("""
Escreva um título para este texto, me dê cinco opções:
Seja ajudando médicos a identificar doenças ou encontrando fotos de “abraços”, a IA está por trás de muito do trabalho que fazemos no Google. E em nosso Arts & Culture Lab em Paris, temos experimentado como a IA pode ser usada em benefício da cultura.
Hoje, estamos compartilhando nossos últimos experimentos - protótipos que se baseiam em sete anos de trabalho em parceria com 1.500 instituições culturais em todo o mundo.
Cada um desses aplicativos experimentais executa algoritmos de IA em segundo plano para permitir que você descubra conexões culturais escondidas em arquivos e até mesmo encontre obras de arte que combinem com a decoração da sua casa."
""", "en")

print(
    traduza(generation_model.predict(prompt, temperature=0.8, max_output_tokens=256, top_k=1, top_p=0.8).text, "pt")
)

## Evaluation
You can evaluate the outputs from summarization tasks using [ROUGE](https://en.wikipedia.org/wiki/ROUGE_(metric)) as an evalulation framework. ROUGE (Recall-Oriented Understudy for Gisting Evaluation) are measures to automatically determine the quality of a summary by comparing it to other (ideal) summaries created by humans. The measures count the number of overlapping units such as n-gram, word sequences, and word pairs between the computer-generated summary to be evaluated and the ideal summaries created by humans.


The first step is to install the ROUGE library.

In [ ]:
!pip install rouge

Create a summary from a language model that you can use to compare against a human-generated summary.

In [ ]:
from rouge import Rouge

ROUGE = Rouge()

prompt = traduza("""
Forneça um resumo muito curto, máximo de quatro frases, para o seguinte artigo:

Nossos computadores quânticos funcionam manipulando qubits de uma forma orquestrada que chamamos de algoritmos quânticos.
O desafio é que os qubits são tão sensíveis que até mesmo a luz difusa pode causar erros de cálculo – e o problema piora à medida que os computadores quânticos crescem.
Isso tem consequências significativas, pois os melhores algoritmos quânticos que conhecemos para executar aplicativos úteis exigem que as taxas de erro de nossos qubits sejam muito menores do que as que temos hoje.
Para preencher essa lacuna, precisaremos de correção de erro quântica.
A correção de erros quânticos protege as informações codificando-as em vários qubits físicos para formar um “qubit lógico” e acredita-se que seja a única maneira de produzir um computador quântico de grande escala com taxas de erro baixas o suficiente para cálculos úteis.
Em vez de calcular nos próprios qubits individuais, calcularemos em qubits lógicos. Ao codificar números maiores de qubits físicos em nosso processador quântico em um qubit lógico, esperamos reduzir as taxas de erro para permitir algoritmos quânticos úteis.

Resumo:

""", "en")

candidate = traduza(generation_model.predict(prompt, temperature=0.1, max_output_tokens=1024, top_k=40, top_p=0.9).text, "pt")

print(candidate)

You will also need a human-generated summary that we will use to compare to the `candidate` generated by the model. We will call this `reference`. 

In [ ]:
reference = "Computadores quânticos são sensíveis a ruídos e erros. Para preencher essa lacuna, precisaremos de correção de erros quânticos. A correção de erros quânticos protege as informações codificando vários qubits físicos para formar um ”qubit lógico”."

Now you can take the candidate and reference to evaluate the performance. In this case, ROUGE will give you:

- `rouge-1`, which measures unigram overlap
- `rouge-2`, which measures bigram overlap
- `rouge-l`, which measures the longest common subsequence

In [ ]:
ROUGE.get_scores(candidate, reference)